In [1]:
from pathlib import Path

In [2]:
# 画像ファイルのリストを取得して、並べ替える
image_paths = sorted(
    Path("datasets/floodnet/FloodNet-Supervised_v1.0/train/train-org-img").glob("*.jpg"),
    key=lambda p: int(p.stem)
)

In [ ]:
len(image_paths)

In [ ]:
# 先頭5件のパスを確認
image_paths[:5]

In [5]:
from IPython.display import display
from PIL import Image

In [ ]:
# 先頭の画像を表示
display(Image.open(image_paths[0]).resize((400,300)))

In [ ]:
image_paths[0]

In [8]:
image_name = image_paths[0].stem

In [ ]:
# 先頭の画像のカラーマスクのパスを確認
colormask_path = Path(r"datasets\floodnet\ColorMasks-FloodNetv1.0\ColorMasks-TrainSet") / f"{image_name}_lab.png"
colormask_path

In [ ]:
# 先頭の画像のカラーマスクを表示
display(Image.open(colormask_path).resize((400,300)))

In [11]:
from tqdm import tqdm

In [12]:
# 各画像のカラーマスクのピクセル値を調べてラベルを決定する関数
def label_from_colormask(image_path):
    colormask_path = Path("datasets/floodnet/ColorMasks-FloodNetv1.0/ColorMasks-TrainSet") / f"{image_path.stem}_lab.png"
    color_mask = Image.open(colormask_path).convert('RGB')
    all_pixels = list(color_mask.getdata())
    unique_colors = [str(color) for color in (set(all_pixels))]

    flood_colors = [
        (255,0,0), # building-flooded
        (160, 150, 20), # road-flooded
    ]
    is_flooded = any(str(colormask) in unique_colors for colormask in flood_colors)
    return "flooded" if is_flooded else "non-flooded"


In [ ]:
# 各画像のラベルを取得
labels = []
for image_path in tqdm(image_paths):
    labels.append(label_from_colormask(image_path))

labels[:5]

In [14]:
import pandas as pd

df = pd.DataFrame({
    'image_path': [str(path) for path in image_paths],
    'label': labels
})

In [ ]:
# ラベルの分布を確認
df['label'].value_counts()

In [16]:
# ラベルを csv へ出力
df.to_csv('labels.csv', index=False)